In [ ]:
# Essential imports for spatial filtering
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import data, filters, feature, restoration, morphology
from scipy import ndimage, signal
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🚀 Ready to explore spatial filtering techniques!")
print(f"OpenCV version: {cv2.__version__}")

# Set random seed for reproducibility
np.random.seed(42)


In [ ]:
# Implement smoothing filters from scratch
def create_average_kernel(size):
    """Create an average (box) filter kernel"""
    return np.ones((size, size)) / (size * size)

def create_gaussian_kernel(size, sigma):
    """Create a Gaussian filter kernel"""
    # Create coordinate matrices
    ax = np.arange(-size // 2 + 1., size // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    
    # Calculate Gaussian values
    kernel = np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(sigma))
    
    # Normalize
    return kernel / np.sum(kernel)

def apply_filter(image, kernel):
    """Apply a filter kernel to an image using convolution"""
    return signal.convolve2d(image, kernel, mode='same', boundary='symm')

def add_noise(image, noise_type='gaussian', amount=0.1):
    """Add different types of noise to an image"""
    if noise_type == 'gaussian':
        noise = np.random.normal(0, amount * 255, image.shape)
        return np.clip(image + noise, 0, 255).astype(np.uint8)
    elif noise_type == 'salt_pepper':
        noisy = image.copy()
        # Salt noise
        salt_coords = tuple([np.random.randint(0, i - 1, int(amount * image.size * 0.5)) 
                            for i in image.shape])
        noisy[salt_coords] = 255
        
        # Pepper noise
        pepper_coords = tuple([np.random.randint(0, i - 1, int(amount * image.size * 0.5)) 
                              for i in image.shape])
        noisy[pepper_coords] = 0
        return noisy
    
    return image

# Load test image
test_image = data.camera()  # Grayscale image

# Add different types of noise
gaussian_noisy = add_noise(test_image, 'gaussian', 0.05)
salt_pepper_noisy = add_noise(test_image, 'salt_pepper', 0.02)

# Create different filter kernels
avg_kernel_3 = create_average_kernel(3)
avg_kernel_7 = create_average_kernel(7)
gauss_kernel_3 = create_gaussian_kernel(3, 1.0)
gauss_kernel_7 = create_gaussian_kernel(7, 2.0)

print("🔧 Smoothing Filter Kernels:")
print("=" * 40)
print("\\n3x3 Average kernel:")
print(avg_kernel_3)
print("\\n3x3 Gaussian kernel (σ=1.0):")
print(gauss_kernel_3.round(4))

# Apply smoothing filters
filters_results = {}

# Test on Gaussian noise
filters_results['gaussian_noise'] = {
    'original': test_image,
    'noisy': gaussian_noisy,
    'avg_3x3': apply_filter(gaussian_noisy, avg_kernel_3),
    'avg_7x7': apply_filter(gaussian_noisy, avg_kernel_7),
    'gauss_3x3': apply_filter(gaussian_noisy, gauss_kernel_3),
    'gauss_7x7': apply_filter(gaussian_noisy, gauss_kernel_7),
    'median_3x3': ndimage.median_filter(gaussian_noisy, size=3),
    'median_7x7': ndimage.median_filter(gaussian_noisy, size=7)
}

# Test on salt-and-pepper noise
filters_results['salt_pepper_noise'] = {
    'original': test_image,
    'noisy': salt_pepper_noisy,
    'avg_3x3': apply_filter(salt_pepper_noisy, avg_kernel_3),
    'avg_7x7': apply_filter(salt_pepper_noisy, avg_kernel_7),
    'gauss_3x3': apply_filter(salt_pepper_noisy, gauss_kernel_3),
    'gauss_7x7': apply_filter(salt_pepper_noisy, gauss_kernel_7),
    'median_3x3': ndimage.median_filter(salt_pepper_noisy, size=3),
    'median_7x7': ndimage.median_filter(salt_pepper_noisy, size=7)
}

# Visualize results
def visualize_smoothing_results(results, noise_type):
    """Visualize smoothing filter results"""
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle(f'Smoothing Filters on {noise_type.title()} Noise', fontsize=14, fontweight='bold')
    
    # First row: Original, Noisy, Average filters
    images_row1 = ['original', 'noisy', 'avg_3x3', 'avg_7x7']
    titles_row1 = ['Original', 'Noisy', 'Average 3x3', 'Average 7x7']
    
    for i, (key, title) in enumerate(zip(images_row1, titles_row1)):
        axes[0, i].imshow(results[key], cmap='gray', vmin=0, vmax=255)
        axes[0, i].set_title(title)
        axes[0, i].axis('off')
    
    # Second row: Gaussian and Median filters
    images_row2 = ['gauss_3x3', 'gauss_7x7', 'median_3x3', 'median_7x7']
    titles_row2 = ['Gaussian 3x3', 'Gaussian 7x7', 'Median 3x3', 'Median 7x7']
    
    for i, (key, title) in enumerate(zip(images_row2, titles_row2)):
        axes[1, i].imshow(results[key], cmap='gray', vmin=0, vmax=255)
        axes[1, i].set_title(title)
        axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize results for both noise types
for noise_type, results in filters_results.items():
    visualize_smoothing_results(results, noise_type.replace('_', ' '))
